In [8]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'aiim-emotion-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F77823%2F8553100%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240529%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240529T111024Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7d64b51623dc44a5088e211860a2586cb5f4f65c4e268c21086797429908df19f4992ad676969a26719c005a2eb8a8876b719c0f1a7573663d9a9e07444cf1edc1c02a11f823e63ce8e17ae95d7e79305fe4c7f8e6efffdd3c4b65870a484c912b593549d83809ecde36f508e440796a311c81b30e2ddee0a842e2e1d1470db287f9050997a7addd702827308275d89b8bfdc6553c36f03dad48df92d95a621f3b52c96c9e69f34e7cab92a4bd1433d83251e7e651be7b88a0c4dc3db9d3c4cfe3aa6c2a341fdc340dd311142039410ca561656b3dcd760319a6783749f0d679709553ee7de36c3484da3cb4e24e792330366099ed7d2b4f1a810c44f39156ba'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 18175099 bytes downloaded
Downloaded and uncompressed: aiim-emotion-classification
Data source import complete.


In [9]:
from keras.utils import image_dataset_from_directory, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomBrightness
from keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import seaborn as sns
import os

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [10]:
base_directory = '/kaggle/input/aiim-emotion-classification/aiim-emotion-classification/'
batch_size = 32
epochs = 20

In [11]:
# Documentation: https://keras.io/api/data_loading/image/
train_ds, validation_ds = image_dataset_from_directory(
    directory=base_directory + 'train/',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True,
    seed=420,
    subset="both",
    validation_split=0.15,
    image_size=(100, 100))


class_names = train_ds.class_names

print(train_ds.class_names)

Found 9108 files belonging to 5 classes.
Using 7742 files for training.
Using 1366 files for validation.
['angry', 'fear', 'happy', 'sad', 'surprise']


## Normalization

In [12]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

data_augmentation = tf.keras.Sequential([
    # RandomFlip("horizontal"),
    # RandomRotation(0.05),
    # RandomZoom(0.05),
    # RandomContrast(0.1),
    # RandomBrightness(0.1)
])
validation_ds = validation_ds.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x), training=True), y))

# Convolutional Operation

## Convolutional Layer 1
* Number of input Channels: 1 (Greyscale)
* Number of output Channels: 32
* Kernel size: 3x3
* Stride: 1
* Padding: 1

## Max Pooling Layer 1
* Kernel size: 2x2
* Stride: 2
* Padding: 0

## Convolutional Layer 2
* Number of input Channels: 32
* Number of output Channels: 64
* kernel size: 3 (3x3 kernel)
* Stride: 1
* Padding 1

## Max Pooling Layer 2
* Kernel size: 2x2
* Stride: 2
* Padding: 0

## Flatten Layer
Converts the multi-dimensional output of the Convolutional and Pooling layers into a 1D vector.

## Fully Connected Layer 1
* Input features: 64 * 25 * 25 (output size after the second pooling layer, flattened)
* Output features: 128

## Fully Connected Layer 2
* Input features: 128
* Output features: Number of classes (5 different emotions)

In [13]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(100,100,1)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),


    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 96, 96, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 48, 48, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                 

Try Learning rate of 0.001

In [14]:
# Define the optimizer
optimizer = Adam(learning_rate=0.001)


# Compile the model
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)



# Training des Modells
history = model.fit(train_ds, validation_data=validation_ds, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, reduce_lr])


Epoch 1/20
242/242 [==============================] - 698s 3s/step - loss: 2.2884 - accuracy: 0.2497 - val_loss: 1.6099 - val_accuracy: 0.3148 - lr: 0.0010
Epoch 2/20
242/242 [==============================] - 675s 3s/step - loss: 1.6051 - accuracy: 0.3397 - val_loss: 1.5351 - val_accuracy: 0.4268 - lr: 0.0010
Epoch 3/20
242/242 [==============================] - 683s 3s/step - loss: 1.5599 - accuracy: 0.3892 - val_loss: 1.4924 - val_accuracy: 0.4627 - lr: 0.0010
Epoch 4/20
242/242 [==============================] - 697s 3s/step - loss: 1.5096 - accuracy: 0.4150 - val_loss: 1.4507 - val_accuracy: 0.4392 - lr: 0.0010
Epoch 5/20
 23/242 [=>............................] - ETA: 9:36 - loss: 1.4804 - accuracy: 0.4321

KeyboardInterrupt: 

In [ ]:
# Vorhersagen für die Validierungsdaten
y_val_true = np.concatenate([y for x, y in validation_ds], axis=0)
y_val_pred = model.predict(validation_ds)

# Umwandeln der Vorhersagen in One-Hot-Format
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true_classes = np.argmax(y_val_true, axis=1)

with np.printoptions(threshold=np.inf):
    print(y_val_pred_classes)

# F1-Score berechnen
f1 = f1_score(y_val_true_classes, y_val_pred_classes, average='weighted')
print("F1-Score: ", f1)

In [ ]:
# Confusion Matrix und Classification Report
cm = confusion_matrix(y_val_true_classes, y_val_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print(classification_report(y_val_true_classes, y_val_pred_classes, target_names=class_names))


In [ ]:
# Plot the loss and accuracy curves for training and validation
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Create Submission

In [ ]:
file_names = []
images = []
test_directory = base_directory + 'test/'

for filename in os.listdir(test_directory):
    if filename.endswith(".jpg"):
        img_path = os.path.join(test_directory, filename)
        img = load_img(img_path, target_size=(100,100), color_mode='grayscale')
        img_array = img_to_array(img)
        img_array = normalization_layer(img_array)
        images.append(img_array)
        file_names.append(filename)

# In ein numpy Array konvertieren
images_array = np.array(images)

# Vorhersagen machen
y_test = model.predict(images_array, batch_size=batch_size)
y_pred_indices = np.argmax(y_test, axis=1)
y_pred_labels = [class_names[idx] for idx in y_pred_indices]

# DataFrame erstellen
df = pd.DataFrame({
    'Id': file_names,
    'emotions': y_pred_labels
})

# DataFrame als CSV speichern
df.to_csv('./submission.csv', index=False)